In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gensim pyarabic emoji

ERROR: Operation cancelled by user


In [ ]:
!pip install numpy==1.26.4

ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import pickle
from tqdm import tqdm
import gensim
from transformers import AutoTokenizer
import nltk
from pyarabic import araby as ar
# from Stemmer import Stemmer
# import Stemmer

# NLTK
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize

KeyboardInterrupt: 

In [ ]:
# Standard Library
# !pip install numpy==1.26.4 pandas==2.2.2 scipy==1.13.1 gensim==4.3.2 tensorflow==2.18.0


import os
import pickle
import re
import string
import emoji
import nltk
import scipy
import numpy as np
import pandas as pd
import gensim
import tensorflow as tf
from pyarabic import araby as ar
# from Stemmer import Stemmer
# import Stemmer

# NLTK
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize

# Scikit-Learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# TensorFlow/Keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Concatenate, Embedding, Input, LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.utils import to_categorical

# Transformers
from transformers import AutoTokenizer

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Science/Arabic Sentiment/Datasets/arabic_sentiment_reviews.csv')
df

,label,content
0,1,النعال المريحة: أرتدي هذه النعال كثيرًا!فهي دا...
1,1,منتج جميل ، خدمة سيئة: لقد اشتريت زوجًا من الن...
2,1,جيد للأشياء الصغيرة: هذا يعمل بشكل جيد لالتقاط...
3,0,واهية للغاية: flimsyif للغاية ، فأنت تشتريه ، ...
4,1,Pop for Girls and Girly Boys ، والأشخاص الذين ...
...,...,...
329995,0,DOA: فتح العلامة التجارية الجديدة من Box.تم تث...
329996,0,شركة صعبة التعامل معها: المنتج كان على ما يرام...
329997,0,SDK Sansa Leather Case: فقير للغاية.لم يتم الإ...
329998,0,حسنًا ، لكن ليس رائعًا: حسنًا ، لقد اشتريت هذا...


In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

# Define Arabic and English punctuations
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations


# Load and filter Arabic stopwords
stop_words = set(stopwords.words('arabic'))
negation_words = {'لا', 'لم', 'لن', 'ما', 'ليس', 'بدون', 'غير'}
stop_words = stop_words.difference(negation_words)


def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def remove_gibberish(text):
    arabic_pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s.,!?؟]')
    return arabic_pattern.sub('', text)

def remove_diacritics(text):
    return ar.strip_tashkeel(text)

def remove_tatweel(text):
    return ar.strip_tatweel(text)

def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

def normalize_elongation(text):
    return re.sub(r'(.)\1+', r'\1\1', text)

def normalize_letters(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)
    return text

def clean_punctuation(text):
    keep = '،.؟!'  # punctuation to keep
    # Replace all punctuation (excluding "keep") with space
    pattern = rf'[{"".join(re.escape(c) for c in punctuations_list if c not in keep)}]'
    text = re.sub(pattern, ' ', text)
    text = re.sub(r'\.{2,}', '.', text)  # Normalize multiple dots
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
    return text

def remove_noise(text):
    text = re.sub(r'http\S+|www.\S+', '', text)  # URLs
    text = re.sub(r'@\w+', '', text)            # Mentions
    text = re.sub(r'#\w+', '', text)            # Hashtags
    text = re.sub(r'\d+', '', text)             # Numbers
    text = re.sub(r'[a-zA-Z]', '', text)        # English letters
    text = re.sub(r'\s+', ' ', text).strip()    # Extra spaces
    return text


def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = text.strip()
    text = remove_emoji(text)
    text = remove_noise(text)
    text = clean_punctuation(text)
    text = remove_gibberish(text)
    text = remove_diacritics(text)
    text = remove_tatweel(text)
    text = normalize_letters(text)
    text = normalize_elongation(text)
    text = remove_stopwords(text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
!apt-get install -y openjdk-8-jdk
!pip install farasapy


In [ ]:
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import pickle
from tqdm import tqdm
import gensim
from transformers import AutoTokenizer


In [ ]:
farasa_segmenter = FarasaSegmenter(interactive=True)
farasa_stemmer = FarasaStemmer(interactive=True)

# Initialize paths
DATA_DIR = "/content/drive/MyDrive/Data Science/Arabic Sentiment/"
MODELS_DIR = os.path.join(DATA_DIR, "Models2/")
FEATURES_DIR = os.path.join(DATA_DIR, "Features/")
DATASETS_DIR = os.path.join(DATA_DIR, "Datasets2/")

# Ensure directories exist
for dir_path in [MODELS_DIR, FEATURES_DIR, DATASETS_DIR]:
    os.makedirs(dir_path, exist_ok=True)


# 1. Function to clean and tokenize text using Farasa
def clean_and_tokenize(texts):
    print("Cleaning and tokenizing texts...")
    cleaned_texts = []
    tokenized_texts = []

    for text in tqdm(texts):
        # Clean first
        cleaned = clean_text(text)
        cleaned_texts.append(cleaned if cleaned else "")

        # Then tokenize using Farasa
        if cleaned:
            # Get segments using Farasa
            segments = farasa_segmenter.segment(cleaned).split()
            tokenized_texts.append(segments)
        else:
            tokenized_texts.append([])

    return cleaned_texts, tokenized_texts
# 3. Function to stem tokenized text using Farasa
def stem_with_farasa(tokenized_texts):
    print("Stemming tokens with Farasa...")
    stemmed_token_lists = []

    for tokens in tqdm(tokenized_texts):
        if tokens:
            # Stem each token individually
            stemmed_tokens = []
            for token in tokens:
                stemmed = farasa_stemmer.stem(token)
                stemmed_tokens.append(stemmed)
            stemmed_token_lists.append(stemmed_tokens)
        else:
            stemmed_token_lists.append([])

    return stemmed_token_lists


In [ ]:
def embed_with_fasttext(fasttext_model_path, stemmed_tokens_path, embedding_dim=300, output_path=None):
    print("Loading stemmed tokens...")
    with open(stemmed_tokens_path, 'rb') as f:
        stemmed_token_lists = pickle.load(f)

    print("Loading FastText model...")
    fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_model_path)

    print("Creating FastText embeddings from stemmed tokens...")
    embeddings = []
    for tokens in tqdm(stemmed_token_lists):
        # Get vectors for tokens that exist in the model
        word_vectors = [fasttext_model[word] for word in tokens if word in fasttext_model]
        if word_vectors:
            # Average the vectors
            text_vector = np.mean(word_vectors, axis=0)
        else:
            # Use zeros if no tokens are in the model
            text_vector = np.zeros(embedding_dim)
        embeddings.append(text_vector)

    embeddings_array = np.array(embeddings)

    # Save embeddings if output path is provided
    if output_path:
        print(f"Saving embeddings to {output_path}")
        np.save(output_path, embeddings_array)

    return embeddings_array

In [ ]:
cleaned_texts, tokenized_texts = clean_and_tokenize(df['content'])
df['cleaned_text'] = cleaned_texts
df['tokenized_text'] = tokenized_texts

In [ ]:
stemmed_token_lists = stem_with_farasa(tokenized_texts)
df['stemmed_tokens'] = stemmed_token_lists

In [ ]:
# Save tokenized data
tokenized_path = os.path.join(FEATURES_DIR, "tokenized_texts.pkl")
with open(tokenized_path, 'wb') as f:
    pickle.dump(tokenized_texts, f)
print(f"Tokenized texts saved to {tokenized_path}")

In [ ]:
# Save stemmed tokens
stemmed_path = os.path.join(FEATURES_DIR, "stemmed_tokens.pkl")
with open(stemmed_path, 'wb') as f:
    pickle.dump(stemmed_token_lists, f)
print(f"Stemmed tokens saved to {stemmed_path}")

In [ ]:
df

In [ ]:
df.to_csv(os.path.join(DATASETS_DIR, "Final_arbic_dataset.csv"), index=False)

In [ ]:
newdata = pd.read_csv('/content/drive/MyDrive/Data Science/Arabic Sentiment/Datasets/stemmed_output.csv')
newdata

-----------------------------------------------------------------

In [ ]:
# Initialize paths
DATA_DIR = "/content/drive/MyDrive/Data Science/Arabic Sentiment/"
MODELS_DIR = os.path.join(DATA_DIR, "Models2/")
FEATURES_DIR = os.path.join(DATA_DIR, "Features/")
DATASETS_DIR = os.path.join(DATA_DIR, "Datasets2/")

# Ensure directories exist
for dir_path in [MODELS_DIR, FEATURES_DIR, DATASETS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

In [ ]:
fasttext_model_path = os.path.join(DATA_DIR, "Models/cc.ar.300.vec.gz")
stemmed_tokens_path = os.path.join(FEATURES_DIR, "stemmed_tokens.pkl")
embeddings_path = os.path.join(FEATURES_DIR, "fasttext_embeddings.npy")

In [ ]:
import os
import numpy as np
import pickle
import gensim
from tqdm import tqdm

# Define the paths
DATA_DIR = "/content/drive/MyDrive/Data Science/Arabic Sentiment/"
FEATURES_DIR = os.path.join(DATA_DIR, "Features/")
fasttext_model_path = os.path.join(DATA_DIR, "Models/cc.ar.300.vec.gz")
stemmed_tokens_path = os.path.join(FEATURES_DIR, "stemmed_tokens.pkl")
embeddings_path = os.path.join(FEATURES_DIR, "fasttext_embeddings.npy")

In [ ]:
# Run the embedding function
X = embed_with_fasttext(
    fasttext_model_path=fasttext_model_path,
    stemmed_tokens_path=stemmed_tokens_path,
    output_path=embeddings_path
)

print(f"FastText embeddings saved to {embeddings_path}")
print(f"Embedding shape: {X.shape}")

In [ ]:
# Cell: Save everything in a single file for easy loading
training_data = {
    'X_fasttext': X
}
training_data_path = os.path.join(FEATURES_DIR, "training_data.pkl")
with open(training_data_path, 'wb') as f:
    pickle.dump(training_data, f)
print(f"Complete training data saved to {training_data_path}")

In [ ]:
with open("/content/drive/MyDrive/Data Science/Arabic Sentiment/Features/training_data.pkl", 'rb') as f:
    training_data = pickle.load(f)

X = training_data['X_fasttext']

In [ ]:
X

array([[-0.02168182, -0.01280909,  0.01896364, ...,  0.02547273,
         0.01144545, -0.0158    ],
       [-0.05181778,  0.00087556, -0.00284   , ...,  0.02858222,
         0.02779778,  0.01011111],
       [-0.0289    , -0.02499091,  0.00394545, ...,  0.02310909,
         0.01787273,  0.00412727],
       ...,
       [-0.05829474,  0.01496842,  0.03438947, ...,  0.01194211,
         0.00835263,  0.02857368],
       [-0.01589444,  0.00213889, -0.00445555, ...,  0.03727778,
         0.02298889, -0.01811111],
       [-0.028195  ,  0.0072925 ,  0.0039625 , ...,  0.0098475 ,
         0.01218   , -0.007785  ]])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y_cat, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Concatenate, Embedding, Input, LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.utils import to_categorical


In [ ]:
best_params = {
    'dense_units1': 256,
    'dense_units2': 64,
    'dropout_rate': 0.3053049220534356,
    'learning_rate': 0.00012779340697800448,
    'optimizer': 'adam',
    'batch_size': 64
}

input_layer = Input(shape=(300,))
dense1 = Dense(best_params['dense_units1'], activation='relu')(input_layer)
dropout1 = Dropout(best_params['dropout_rate'])(dense1)
dense2 = Dense(best_params['dense_units2'], activation='relu')(dropout1)
dropout2 = Dropout(best_params['dropout_rate'])(dense2)
output = Dense(y_train.shape[1], activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output)

optimizer = Adam(learning_rate=best_params['learning_rate'])
model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
print("Training best model with parameters:")
for k, v in best_params.items():
    print(f"{k}: {v}")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=best_params['batch_size'],
    callbacks=[early_stop],
    verbose=1
)

# Save the model (optional)
save_dir = '/content/drive/MyDrive/Data Science/Arabic Sentiment/Models'
os.makedirs(save_dir, exist_ok=True)
model_save_path = os.path.join(save_dir, 'best_training_modelTEST.h5')
model.save(model_save_path)
print(f"\nModel saved to {model_save_path}")

# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.4f}")

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Generate classification report
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes))

Training best model with parameters:
dense_units1: 256
dense_units2: 64
dropout_rate: 0.3053049220534356
learning_rate: 0.00012779340697800448
optimizer: adam
batch_size: 64
Epoch 1/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.7163 - loss: 0.5523 - val_accuracy: 0.7948 - val_loss: 0.4473
Epoch 2/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 45s 7ms/step - accuracy: 0.7880 - loss: 0.4556 - val_accuracy: 0.7993 - val_loss: 0.4350
Epoch 3/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - accuracy: 0.7961 - loss: 0.4430 - val_accuracy: 0.8031 - val_loss: 0.4280
Epoch 4/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.7995 - loss: 0.4360 - val_accuracy: 0.8014 - val_loss: 0.4274
Epoch 5/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.8044 - loss: 0.4287 - val_accuracy: 0.8081 - val_loss: 0.4169
Epoch 6/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 39s 5ms/step - accuracy: 0.8070 - loss: 0.4221 - val_accuracy: 0.8102 - val_loss: 0.4137
Epoch 7/20
3713/3713 ━━━━━━━━━━━━━━━━━━━


Model saved to /content/drive/MyDrive/Data Science/Arabic Sentiment/Models/best_training_modelTEST.h5

Test accuracy: 0.8195
2063/2063 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.83      0.82     32629
           1       0.83      0.81      0.82     33371

    accuracy                           0.82     66000
   macro avg       0.82      0.82      0.82     66000
weighted avg       0.82      0.82      0.82     66000



In [ ]:
!pip install optuna

In [ ]:
import optuna
def objective(trial):
    dense_units = trial.suggest_categorical("dense_units", [32, 64, 128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop", "nadam"])
    batch_size = trial.suggest_categorical("batch_size", [16, 32])

    input_layer = Input(shape=(X_train.shape[1],))  # Use actual vector shape
    dense = Dense(dense_units, activation='relu')(input_layer)
    dropout = Dropout(dropout_rate)(dense)
    output = Dense(1, activation='sigmoid')(dropout)

    model = Model(inputs=input_layer, outputs=output)

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)

    history = model.fit(
        X_train, y_train[:, 1],
        validation_data=(X_val, y_val[:, 1]),
        epochs=5,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=1
    )

    return max(history.history["val_accuracy"])


# ========== Run Optuna Study ==========
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# ========== Best Trial Results ==========
print("Best trial:")
trial = study.best_trial
for key, value in trial.params.items():
    print(f"{key}: {value}")

In [ ]:
# Load the saved model
from tensorflow.keras.models import load_model
model_path = '/content/drive/MyDrive/Data Science/Arabic Sentiment/Models/best_training_model.h5'
model = load_model(model_path)

# Define unlabeled test cases - mix of MSA and dialects
test_cases = [
    # Modern Standard Arabic
    "الفيلم كان رائعاً من جميع النواحي، التمثيل والإخراج والموسيقى التصويرية",
    "الخدمة كانت سيئة للغاية ولا أنصح أحداً بهذا المكان",

    # Egyptian Arabic
    "الاكل كان لذيذ جدا والخدمه سريعه والنظافه ممتازه",
    "المكان مش نظيف والموظفين مش متعاونين خالص",

    # Levantine Arabic
    "المطعم كتير حلو والاكل طيب ونظيف",
    "الخدمة سيئة وما في اي التزام بالمواعيد",

    # Gulf Arabic
    "الخدمة ممتازة والاكل طيب والجو حلو",
    "المكان مو نظيف والجو مزعج",

    # Mixed dialect with MSA
    "التجربة كانت جميلة جداً والستاف متعاون كتير",
    "المنتج مش شغال كويس وهو ليس كما وصف",

    # Emoji and slang cases
    "المنتج رائع 😍😍 يستحق التجربة",
    "سيء جدا 😡 ما انصح فيه ابدا",

    # Short cases
    "حلو",
    "سيء",

    # Neutral/ambiguous cases
    "المنتج مقبول ولكن السعر مرتفع بعض الشيء",
    "الخدمة متوسطة ليست سيئة ولا ممتازة",

    # Code-switching with English
    "المنتج amazing والجودة top",
    "الservice كان slow جداً",

    # Arabizi (Arabic written in Latin script)
    "el mawdoo3 kwayes awi",  # الموضوع كويس قوي
    "mabsoot 3ala el service",  # مبسوط على السيرفيس

    # URLs and mentions (to test preprocessing)
    "الموقع جميل https://example.com @username",
    "تابعني على تويتر @user123 للعروض الجديدة",

    # Numbers and symbols
    "اشتريت 3 قطع وجميعها معيبة",
    "السعر $$$ لكن الجودة ##",

    # Long text
    "بعد تجربة طويلة مع هذا المنتج يمكنني القول أنه يستحق الشراء بكل تأكيد، الجودة عالية جداً والتغليف ممتاز، ولكن هناك بعض الملاحظات البسيطة على دليل الاستخدام الذي يحتاج إلى مزيد من التوضيح",

    # Very short text
    "لا"
]

# Preprocess the test cases using your existing functions
print("Cleaning and tokenizing test cases...")
cleaned_test, tokenized_test = clean_and_tokenize(test_cases)
stemmed_test = stem_with_farasa(tokenized_test)

# Modified embedding function that can accept either a file path or direct tokens
def embed_with_fasttext(fasttext_model_path, stemmed_tokens=None, stemmed_tokens_path=None, embedding_dim=300, output_path=None):
    if stemmed_tokens is None:
        if stemmed_tokens_path is None:
            raise ValueError("Either stemmed_tokens or stemmed_tokens_path must be provided")
        print("Loading stemmed tokens from file...")
        with open(stemmed_tokens_path, 'rb') as f:
            stemmed_token_lists = pickle.load(f)
    else:
        print("Using provided stemmed tokens...")
        stemmed_token_lists = stemmed_tokens

    print("Loading FastText model...")
    fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_model_path)

    print("Creating FastText embeddings from stemmed tokens...")
    embeddings = []
    for tokens in tqdm(stemmed_token_lists):
        # Get vectors for tokens that exist in the model
        word_vectors = [fasttext_model[word] for word in tokens if word in fasttext_model]
        if word_vectors:
            # Average the vectors
            text_vector = np.mean(word_vectors, axis=0)
        else:
            # Use zeros if no tokens are in the model
            text_vector = np.zeros(embedding_dim)
        embeddings.append(text_vector)

    embeddings_array = np.array(embeddings)

    # Save embeddings if output path is provided
    if output_path:
        print(f"Saving embeddings to {output_path}")
        np.save(output_path, embeddings_array)

    return embeddings_array

# Now run the test cases with the corrected function
print("\nCreating embeddings...")
test_embeddings = embed_with_fasttext(
    fasttext_model_path=fasttext_model_path,
    stemmed_tokens=stemmed_test,  # Pass the tokens directly
    embedding_dim=300
)

# The rest of your code remains the same...
print("\nMaking predictions...")
predictions = model.predict(test_embeddings)
predicted_labels = np.argmax(predictions, axis=1)
confidence_scores = np.max(predictions, axis=1)

# Display results with confidence scores
print("\nPredictions:")
for i, (text, label, confidence) in enumerate(zip(test_cases, predicted_labels, confidence_scores)):
    sentiment = "Positive" if label == 1 else "Negative"
    print(f"Case {i+1}:")
    print(f"Original: {text}")
    print(f"Cleaned: {cleaned_test[i]}")
    print(f"Prediction: {sentiment} (Confidence: {confidence:.2f})")
    print("-----")